In [1]:
import numpy as np
from utils.slam_helpers import transform_to_frame,transformed_params2depthplussilhouette,transformed_params2rendervar,transformed_GRNparams2rendervar,transformed_GRNparams2depthplussilhouette
from diff_gaussian_rasterization import GaussianRasterizer as Renderer
from scripts.main_SurgeSplat import deform_gaussians, setup_camera
import torch
import os
from PIL import Image

System Paths:
/media/thesis_ssd/code/EndoGSLAM/EndoGSLAM
/media/thesis_ssd/anaconda/envs/endogslam/lib/python310.zip
/media/thesis_ssd/anaconda/envs/endogslam/lib/python3.10
/media/thesis_ssd/anaconda/envs/endogslam/lib/python3.10/lib-dynload

/media/thesis_ssd/anaconda/envs/endogslam/lib/python3.10/site-packages


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [ ]:
intrinsics = torch.tensor([[199.6883,   0.0000, 166.3290],
        [  0.0000, 249.4753, 170.4058],
        [  0.0000,   0.0000,   1.0000]], device='cuda:0')
w2c = torch.tensor([[ 1.0000e+00,  6.5711e-11,  2.3283e-10,  0.0000e+00],
        [-3.1832e-11,  1.0000e+00, -7.4115e-21,  0.0000e+00],
        [-9.2644e-22,  2.9104e-11,  1.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]], device='cuda:0')

cam = setup_camera(336,336, intrinsics.cpu().numpy(), w2c.detach().cpu().numpy(), use_simplification=True)




In [ ]:
params_np = np.load('/media/thesis_ssd/code/EndoGSLAM/EndoGSLAM/experiments/EndoNerf cutting_deform_short_simple/cutting_deform_short_simple/params.npz')
params={}
for key in params_np.keys():
    params[key] = torch.tensor(params_np[key]).cuda()

In [ ]:
params

{'means3D': tensor([[[-6.8815, -6.8403, -6.8349,  ..., -6.8815, -6.8815, -6.8815],
          [-5.6432, -5.6250, -5.6242,  ..., -5.6432, -5.6432, -5.6432],
          [ 8.2617,  8.2973,  8.2960,  ...,  8.2617,  8.2617,  8.2617]],
 
         [[-6.6291, -6.5871, -6.6702,  ..., -6.6291, -6.6291, -6.6291],
          [-5.4691, -5.4067, -5.4271,  ..., -5.4691, -5.4691, -5.4691],
          [ 8.0068,  8.0823,  8.0451,  ...,  8.0068,  8.0068,  8.0068]],
 
         [[-6.5513, -6.5090, -6.5918,  ..., -6.5513, -6.5513, -6.5513],
          [-5.4378, -5.3838, -5.3943,  ..., -5.4378, -5.4378, -5.4378],
          [ 7.9609,  8.0409,  8.0013,  ...,  7.9609,  7.9609,  7.9609]],
 
         ...,
 
         [[ 5.7406,  5.7791,  5.7877,  ...,  5.7406,  5.7406,  5.7406],
          [ 4.5106,  4.5159,  4.4997,  ...,  4.5106,  4.5106,  4.5106],
          [ 6.8368,  6.8239,  6.8307,  ...,  6.8368,  6.8368,  6.8368]],
 
         [[ 6.1185,  6.1092,  6.1295,  ...,  6.1185,  6.1185,  6.1185],
          [ 4.7790,  4.77

In [ ]:

# def deform_gaussians(params,time,deform_grad):

#     if deform_grad:
#         weights = params['deform_weights']
#         stds = params['deform_stds']
#         biases = params['deform_biases']
#     else:
#         weights = params['deform_weights'].detach()
#         stds = params['deform_stds'].detach()
#         biases = params['deform_biases'].detach()

#     deform = torch.sum(weights*torch.exp(-1/(2*stds**2)*(time-biases)**2),1) # Nx10 gaussians deformations
#     deform_xyz = deform[:,:3]
#     deform_rots = deform[:,3:7]
#     deform_scales = deform[:,7:10]
#     # print(f'xyz: {torch.sum(deform_xyz)}')
#     # print(torch.sum(deform_rots).item())
#     # print(torch.sum(deform_scales).item())
#     xyz = params['means3D']+deform_xyz
#     rots = params['unnorm_rotations']+deform_rots
#     scales = params['log_scales']+deform_scales

#     return xyz,rots,scales


# def deform_gaussians(params, time, deform_grad, N=5):
#     """
#     Calculate deformations using the N closest basis functions based on |time - bias|.

#     Args:
#         params (dict): Dictionary containing deformation parameters.
#         time (torch.Tensor): Current time step.
#         deform_grad (bool): Whether to calculate gradients for deformations.
#         N (int): Number of closest basis functions to consider.

#     Returns:
#         xyz (torch.Tensor): Updated 3D positions.
#         rots (torch.Tensor): Updated rotations.
#         scales (torch.Tensor): Updated scales.
#     """
#     if deform_grad:
#         weights = params['deform_weights']
#         stds = params['deform_stds']
#         biases = params['deform_biases']
#     else:
#         weights = params['deform_weights'].detach()
#         stds = params['deform_stds'].detach()
#         biases = params['deform_biases'].detach()

#     # Calculate the absolute difference between time and biases
#     time_diff = torch.abs(time - biases)

#     # Get the indices of the N smallest time differences
#     _, top_indices = torch.topk(-time_diff, N, dim=1)  # Negative for smallest values

#     # Create a mask to select only the top N basis functions
#     mask = torch.zeros_like(time_diff, dtype=torch.float)
#     mask.scatter_(1, top_indices, 1.0)

#     # Apply the mask to weights and biases
#     masked_weights = weights * mask
#     masked_biases = biases * mask

#     # Calculate deformations
#     deform = torch.sum(
#         masked_weights * torch.exp(-1 / (2 * stds**2) * (time - masked_biases)**2), dim=1
#     )  # Nx10 gaussians deformations

#     deform_xyz = deform[:, :3]
#     deform_rots = deform[:, 3:7]
#     deform_scales = deform[:, 7:10]

#     xyz = params['means3D'] + deform_xyz
#     rots = params['unnorm_rotations'] + deform_rots
#     scales = params['log_scales'] + deform_scales

#     return xyz, rots, scales

def deform_gaussians(params, time, deform_grad, N=5,deformation_type = 'gaussian'):
    """
    Calculate deformations using the N closest basis functions based on |time - bias|.

    Args:
        params (dict): Dictionary containing deformation parameters.
        time (torch.Tensor): Current time step.
        deform_grad (bool): Whether to calculate gradients for deformations.
        N (int): Number of closest basis functions to consider.

    Returns:
        xyz (torch.Tensor): Updated 3D positions.
        rots (torch.Tensor): Updated rotations.
        scales (torch.Tensor): Updated scales.
    """
    if deformation_type =='gaussian':
        if True:
            if deform_grad:
                weights = params['deform_weights']
                stds = params['deform_stds']
                biases = params['deform_biases']
            else:
                weights = params['deform_weights'].detach()
                stds = params['deform_stds'].detach()
                biases = params['deform_biases'].detach()

            # Calculate the absolute difference between time and biases
            time_diff = torch.abs(time - biases)

            # Get the indices of the N smallest time differences
            _, top_indices = torch.topk(-time_diff, N, dim=1)  # Negative for smallest values

            # Create a mask to select only the top N basis functions
            mask = torch.zeros_like(time_diff, dtype=torch.float)
            mask.scatter_(1, top_indices, 1.0)

            # Apply the mask to weights and biases
            masked_weights = weights * mask
            masked_biases = biases * mask

            # Calculate deformations
            deform = torch.sum(
                masked_weights * torch.exp(-1 / (2 * stds**2) * (time - masked_biases)**2), dim=1
            )  # Nx10 gaussians deformations

            deform_xyz = deform[:, :3]
            deform_rots = deform[:, 3:7]
            deform_scales = deform[:, 7:10]
        else:
            
            if deform_grad:
                weights = params['deform_weights']
                stds = params['deform_stds']
                biases = params['deform_biases']
            else:
                weights = params['deform_weights'].detach()
                stds = params['deform_stds'].detach()
                biases = params['deform_biases'].detach()

            # Calculate the absolute difference between time and biases
            time_diff = torch.abs(time - biases)

            # Get the indices of the N smallest time differences
            _, top_indices = torch.topk(-time_diff, N, dim=1)  # Negative for smallest values

            # Create a mask to select only the top N basis functions
            mask = torch.zeros_like(time_diff, dtype=torch.float)
            mask.scatter_(1, top_indices, 1.0).detach()

            # Register a gradient hook to zero out gradients for irrelevant basis functions
            if deform_grad:
                def zero_out_irrelevant_gradients(grad):
                    return grad * mask

                weights.register_hook(zero_out_irrelevant_gradients)
                biases.register_hook(zero_out_irrelevant_gradients)
                stds.register_hook(zero_out_irrelevant_gradients)

            # Calculate deformations
            deform = torch.sum(
                weights * torch.exp(-1 / (2 * stds**2) * (time - biases)**2), dim=1
            )  # Nx10 gaussians deformations

            deform_xyz = deform[:, :3]
            deform_rots = deform[:, 3:7]
            deform_scales = deform[:, 7:10]

        xyz = params['means3D'] + deform_xyz
        rots = params['unnorm_rotations'] + deform_rots
        scales = params['log_scales'] + deform_scales


    elif deformation_type == 'simple':
        with torch.no_grad():
            xyz = params['means3D'][:,:,time]
            rots = params['unnorm_rotations'][:,:,time]
            scales = params['log_scales'][:,:,time]

    return xyz, rots, scales

for id in range(params['cam_unnorm_rots'].shape[-1]):
    local_means,local_rots,local_scales = deform_gaussians(params,id,deform_grad = True,deformation_type='simple')


    #  print(torch.sum(local_means-params['means3D']))

    transformed_pts = transform_to_frame(local_means,params,id,False,False)





    # Initialize Render Variables
    rendervar = transformed_GRNparams2rendervar(params, transformed_pts,local_rots,local_scales)
    print(local_scales.max())
    rv_store = {}
    for key in rendervar.keys():
        rv_store[key] = rendervar[key].cpu().detach()
        local_means_store = local_means.cpu()
        local_scales_store = local_rots.cpu()
        local_rots_store = local_rots.cpu()
        transformed_pts_store = transformed_pts.cpu()



    #  rendervar['means3D'].retain_grad()
    depth_sil_rendervar = transformed_GRNparams2depthplussilhouette(params, w2c,
                                            transformed_pts,local_rots,local_scales)


    #RGB Rendering

    rendervar['means2D'].retain_grad()
    im, radius, _ = Renderer(raster_settings=cam)(**rendervar)
    # variables['means2D'] = rendervar['means2D'] # Gradient only accum from colour render for densification
    img = Image.fromarray((im.permute(1,2,0).cpu().detach().numpy()*255).astype(np.uint8))
    os.makedirs(f'./eval_plots/plots_simple/',exist_ok=True)
    img.save(f'./eval_plots/plots_simple/{id}.png')

    print(local_means)


tensor(0.0002, device='cuda:0')
tensor([[-6.8815, -5.6432,  8.2617],
        [-6.6291, -5.4691,  8.0068],
        [-6.5513, -5.4378,  7.9609],
        ...,
        [ 5.7406,  4.5106,  6.8368],
        [ 6.1185,  4.7790,  7.2436],
        [ 3.9109,  1.9012,  5.6123]], device='cuda:0')
tensor(0.0002, device='cuda:0')
tensor([[-6.8403, -5.6250,  8.2973],
        [-6.5871, -5.4067,  8.0823],
        [-6.5090, -5.3838,  8.0409],
        ...,
        [ 5.7791,  4.5159,  6.8239],
        [ 6.1092,  4.7756,  7.2506],
        [ 3.9109,  1.9012,  5.6123]], device='cuda:0')
tensor(0.0002, device='cuda:0')
tensor([[-6.8349, -5.6242,  8.2960],
        [-6.6702, -5.4271,  8.0451],
        [-6.5918, -5.3943,  8.0013],
        ...,
        [ 5.7877,  4.4997,  6.8307],
        [ 6.1295,  4.7894,  7.2339],
        [ 3.9109,  1.9012,  5.6123]], device='cuda:0')
tensor(0.0002, device='cuda:0')
tensor([[-6.8753, -5.5897,  8.2926],
        [-6.6383, -5.4661,  8.0177],
        [-6.5889, -5.3962,  7.9605],
  

In [ ]:
params['means3D'][...,0].shape

torch.Size([112897, 3])

In [ ]:
# import matplotlib.pyplot as plt
# weights =   params['deform_weights'].cpu().detach()
# biases =    params['deform_biases'].cpu().detach()
# stds =      params['deform_stds'].cpu().detach()

# deforms = []
# deformsx1 = []
# for time in range(100):
#     deform = torch.sum(weights*torch.exp(-1/(2*stds**2)*(time-biases)**2),1)
#     deforms.append(deform) # Nx10 gaussians deformations
#     deformsx1.append(deform[0,0])
# fig,ax = plt.subplots(10,10,figsize = (25,25),sharey=True)
# for i in range(10):
#     for ii in range(10):
#         ax[ii,i].plot([deforms[idx][i+91720//3,ii] for idx in range(len(deforms))])
#         ax[ii,i].axhline(y = 0.0,color = 'r', linestyle = '--')    

In [ ]:
# from GRN.models.conv_unet import GaussianRegressionNetwork
# import torch

# state_dict = torch.load('logs/GRN_6/checkpoint.pth',weights_only=False)
# model_state_dict = state_dict['model']

# model = GaussianRegressionNetwork()
# # model.load_state_dict(model_state_dict)

# torch.save(model.state_dict(),'GRN/models/GRN_v1.pth')
